In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
# np.random.seed(20)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, roc_auc_score, plot_roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/employee-attrition/HR-Employee-Attrition.csv"
)

In [ ]:
df_copy = df.copy()

In [ ]:
df.head()

# > **EDA**

In [ ]:
df.info()

In [ ]:
# From above information, we can get to know that there are no null values in any of the columns.

In [ ]:
df['Age'].value_counts()

In [ ]:
df['Attrition'].value_counts()

In [ ]:
df['Attrition'] = np.where(df['Attrition']=='Yes', 1,0)

In [ ]:
df['Attrition'].value_counts()

In [ ]:
df.head()

In [ ]:
# We can see that data is imbalanced since for 'Yes' its only 237 and '1233' for NO. We will check on this later.

In [ ]:
df['BusinessTravel'].value_counts()

In [ ]:
plt.figure(figsize=(7,7))
ax = sns.countplot(x='Attrition', hue='BusinessTravel', data=df)
plt.title('Distribution of Truck Configurations')
plt.xlabel('Attrition')
plt.ylabel('BusinessTravel')

for p in ax.patches:
        ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+20))

In [ ]:
# ppl who travel rarely are less likely to attrition

In [ ]:
df['DailyRate'].value_counts()

In [ ]:
plt.figure(figsize=(16,8))

ax = sns.barplot(x='JobRole', y='DailyRate', hue='Attrition', data=df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

In [ ]:
# We see that healthcare representatives earn the most and managers are more likely to be attrited 

In [ ]:
df['Department'].value_counts()

In [ ]:
sns.barplot(x='Department', y='DistanceFromHome', data=df)

In [ ]:
# The Sales department members travel the most

In [ ]:
sns.barplot(x="Education", y='JobRole', data=df)

In [ ]:
# Employees as Research Directors have the max number of degress to their name

In [ ]:
df.head()

In [ ]:
df['DistanceFromHome'].value_counts()

In [ ]:
# sns.barplot(x='DistanceFromHome', y='Attrition', data=df)

In [ ]:
df['Education'].value_counts()

In [ ]:
df['EducationField'].value_counts()

In [ ]:
sns.countplot(x="EducationField", hue='Attrition', data=df)

In [ ]:
# The rate of attrition is highest for employees under LifeSciences and least for HR employees

In [ ]:
df['EmployeeCount'].value_counts()

In [ ]:
# df = df.drop(columns=['EmployeeCount'])

In [ ]:
df.head()

In [ ]:
df['Gender'].value_counts()

In [ ]:
# df['Gender'] = np.where(df['Gender']=='Male', 0, 1)

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['HourlyRate'].value_counts()

In [ ]:
df.head()

In [ ]:
df['MaritalStatus'].value_counts()

In [ ]:
df['MonthlyIncome'].value_counts()

In [ ]:
sns.scatterplot(x='StockOptionLevel', y='MonthlyIncome', data=df)

In [ ]:
# Top Stock option is NOT so widely provided for employees with high salary

In [ ]:
df['JobInvolvement'].value_counts()

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.countplot(hue='Attrition', x='TotalWorkingYears', data=df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

In [ ]:
# Employees with less experience have higher chance of attrition

In [ ]:
df['JobRole'].value_counts()

In [ ]:
sns.countplot(x='YearsSinceLastPromotion', hue='Attrition', data=df)

In [ ]:
df['JobSatisfaction'].value_counts()

In [ ]:
df['NumCompaniesWorked'].value_counts()

In [ ]:
df.head()

In [ ]:
df['Over18'].value_counts()

In [ ]:
# df = df.drop(columns = ['Over18'])

In [ ]:
df['OverTime'].value_counts()

In [ ]:
# df['OverTime'] = np.where(df['OverTime']=='Yes', 1, 0)

In [ ]:
df['OverTime'].value_counts()

In [ ]:
df['StandardHours'].value_counts()

In [ ]:
# df.drop(columns=['StandardHours'], inplace=True)

In [ ]:
df['EmployeeNumber'].value_counts()

In [ ]:
# df.drop(columns=['EmployeeNumber'], inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

# Feature Engineering

In [ ]:
df.corr()

In [ ]:
y = df['Attrition']

In [ ]:
df.head()

In [ ]:
x = df.drop(columns=['Attrition', 'EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])

In [ ]:
df_numeric_cols = x.select_dtypes(include=np.number)

In [ ]:
df_numeric_cols.head()

In [ ]:
df.columns.difference(df_numeric_cols.columns)

In [ ]:
# df_numeric_cols.head()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = pd.DataFrame()

In [ ]:
vif['Features'] = df_numeric_cols.columns

In [ ]:
vif['VIF'] = [variance_inflation_factor(df_numeric_cols.values, i) for i in range(len(df_numeric_cols.columns))]

In [ ]:
vif

In [ ]:
# We will try to remove the features with high variance after running the model with all the features first
# Then we can compare the performance after removing each feature with high variance

# Encoding categorical data

In [ ]:
x['Gender'] = np.where(x['Gender'] == 'Male', 0, 1)

In [ ]:
df_cat_cols = x.select_dtypes(include='object')

In [ ]:
df_cat_cols.head()

In [ ]:
len(df_cat_cols.columns)

In [ ]:

df_cat_encoded = pd.get_dummies(df_cat_cols)

In [ ]:
df_cat_encoded.head()

In [ ]:
x_final = pd.concat([df_numeric_cols, df_cat_encoded], axis=1)

In [ ]:
x_final.head()

In [ ]:
x_final.shape

# Scaling

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_x = scaler.fit_transform(x_final)

In [ ]:
scaled_x_df = pd.DataFrame(data=scaled_x, columns=x_final.columns)

In [ ]:
scaled_x_df.head()

In [ ]:
trainX, testX, trainY, testY = train_test_split(scaled_x_df,y,test_size=0.3, random_state=220, stratify=y)

In [ ]:
trainX.head()

In [ ]:
trainY.head()

# Logistic Regression

In [ ]:
def model_fit(model, testX, testY, trainX, trainY, cv=True, cv_folds=10):
    model.fit(trainX, trainY)
    predictY = model.predict(testX)
    if cv:
        cv_score = cross_val_score(model, trainX, trainY, cv=cv_folds, scoring='f1_macro')
    
    print("CV report: Mean %.3g| Std %.3g| Min %.3g | Max %.3g"%((np.mean(cv_score), 
                                                                  np.std(cv_score), 
                                                                  np.min(cv_score), 
                                                                  np.max(cv_score))))
    print("Classification Report:\n",classification_report(testY, predictY))
    print("Confusion Matrix:\n", confusion_matrix(testY, predictY))
    return predictY
    
    

In [ ]:
lr = LogisticRegression()
predictY = model_fit(lr, testX, testY, trainX, trainY)
# plot_roc_curve(lr, trainX, trainY)

In [ ]:
# Getting a recall of 1, need to check

In [ ]:
auc = roc_auc_score(testY, predictY)
print('auc score {}'.format(auc))
fpr, tpr, thresholds = roc_curve(testY, predictY)
plt.plot(fpr, tpr)

# KNN

In [ ]:
def knn_model(trainX, trainY, testX, testY, k_value=5, weights='distance', metric='manhattan'):
#     scaling = StandardScaler()
#     train_scaled = scaling.fit_transform(trainX)
#     test_scaled =  scaling.transform(testX)
    knn = KNeighborsClassifier(n_neighbors=k_value, weights=weights, metric=metric, n_jobs=-1)
    knn.fit(trainX, trainY)
    trainpredictY = knn.predict(trainX)
    testpredictY = knn.predict(testX)
    return knn, trainpredictY, testpredictY
    

In [ ]:
for k_values in range(3,11):
    print("For k value of {}".format(k_values))
    knn, trainpredictY, testpredictY = knn_model(trainX, trainY, testX, testY, k_values)
    print("Classification report for train\n", classification_report(trainY, trainpredictY))
    print("\n\n")
    print("Classification report for test\n", classification_report(testY, testpredictY))
    print("\n\n")
    
    

In [ ]:
# We can select the k value of 5 since it provides the highest accuracy of 86

In [ ]:
for weights in ['uniform', 'distance']:
    print("For weights as {}".format(weights))
    knn, trainpredictY, testpredictY = knn_model(trainX, trainY, testX, testY, weights=weights)
    print("Classification Report for train\n", classification_report(trainY, trainpredictY))
    print("\n\n")
    print("Classification Report for test\n", classification_report(testY, testpredictY))
    print("\n\n")
    

In [ ]:
# Both uniform and distance weights provide the same accuracy, we will go with distance

In [ ]:
for metrics in ["manhattan", "chebyshev", "minkowski", "euclidean"]:
    print("For metric {}\n".format(metrics))
    knn, trainpredictY, testpredictY = knn_model(trainX, trainY, testX, testY, metric=metrics)
    print("Classification Report for train\n", classification_report(trainY, trainpredictY))
    print("\n\n")
    print("Classification Report for test\n", classification_report(testY, testpredictY))
    print("\n\n")

In [ ]:
# Manhattan is the metric to be chosen, from the accuracy score obtained.

In [ ]:
knn, trainpredictY, testpredictY = knn_model(trainX, trainY, testX, testY, k_value=7, weights='distance', metric='manhattan')
print("Classification Report for trained data\n", classification_report(trainY, trainpredictY))
print("Confusion Matrix for\n", confusion_matrix(trainY, trainpredictY))
print("Classification Report for test data\n", classification_report(testY, testpredictY))
print("Confusion Matrix for\n", confusion_matrix(trainY, trainpredictY))
testY_re = np.array(testY).reshape(-1,1)
# testpredictY_re = np.array(testpredictY).reshape(-1,1)
auc = roc_auc_score(testY, testpredictY)
print('auc score {}'.format(auc))
fpr, tpr, thresholds = roc_curve(testY, testpredictY)
plt.plot(fpr, tpr)
# fpr_re = fpr.reshape(-1,1)
# tpr_re = tpr.reshape(-1,1)
# plot_roc_curve(knn, fpr_re, tpr_re)


In [ ]:
# We need to remove few features that are highly correlated since we are getting a hight recall of 1.0

# Naive Bayes


In [ ]:
nb = GaussianNB()
nb.fit(trainX, trainY)
trainpredictY = nb.predict(trainX)
testpredictY = nb.predict(testX)
print("Confusion Matrix for train data \n", confusion_matrix(trainY, trainpredictY))
print("Classification Report for train data\n", classification_report(trainY, trainpredictY))
print("Confusion Matrix for test data \n", confusion_matrix(testY, testpredictY))
print("Classification Report for test data\n", classification_report(testY, testpredictY))
trainY = np.array(trainY).reshape(-1,1)
trainpredictY = np.array(trainpredictY).reshape(-1,1)
testY = np.array(testY).reshape(-1,1)
testpredictY = np.array(testpredictY).reshape(-1,1)

auc = roc_auc_score(testY, testpredictY)
print('auc score {}'.format(auc))
fpr, tpr, thresholds = roc_curve(testY, testpredictY)
plt.plot(fpr, tpr)


# Using Logistic Regression with higher degrees

In [ ]:
def poly(trainX, trainY, degrees):
    poly = PolynomialFeatures(degree=degrees)
    train_poly = poly.fit_transform(trainX)
    test_poly = poly.fit_transform(testX)
    return train_poly, test_poly

In [ ]:
for degrees in [1,2,3]:
    print("For degree {}".format(degrees))
    train_poly, test_poly = poly(trainX, trainY, degrees)
    lr = LogisticRegression()
    model_fit(lr, test_poly, testY, train_poly, trainY)


In [ ]:
# No Degrees are required, since degree 1 gave best results in terms of recall, precision, accuracy

In [ ]:
# NO change in accuracy observed with increase in polynomial

# Using Smote with Logisitc regression, KNN, Gausian NB

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=5)
print(trainX.shape)
print(trainY.shape)
trainX_oversampled, trainY_oversampled = sm.fit_resample(trainX, trainY)


print("The training data is changed to \n")

print((pd.DataFrame(data=trainY, columns=['Attrition'])).value_counts())
print((pd.DataFrame(data=trainY_oversampled, columns=['Attrition'])).value_counts())

In [ ]:
lr = LogisticRegression()
predictY = model_fit(lr, testX, testY, trainX_oversampled, trainY_oversampled)
auc = roc_auc_score(testY, predictY)
print('auc score {}'.format(auc))
fpr, tpr, thresholds = roc_curve(testY, predictY)
plt.plot(fpr, tpr)

In [ ]:
# The output from the Smote looks more realistic since 863 entries for Attrition1 and 0 are taken to train the Regression Model

In [ ]:
knn, trainpredictY, testpredictY = knn_model(trainX_oversampled, trainY_oversampled, testX, testY, k_value=7, weights='distance', metric='manhattan')
print("Classification Report for trained data\n", classification_report(trainY_oversampled, trainpredictY))
print("Confusion Matrix for\n", confusion_matrix(trainY_oversampled, trainpredictY))
print("Classification Report for test data\n", classification_report(testY, testpredictY))
print("Confusion Matrix for\n", confusion_matrix(testY, testpredictY))
testY_re = np.array(testY).reshape(-1,1)
# testpredictY_re = np.array(testpredictY).reshape(-1,1)
auc = roc_auc_score(testY, testpredictY)
print('auc score {}'.format(auc))
fpr, tpr, thresholds = roc_curve(testY, testpredictY)
plt.plot(fpr, tpr)

From the above analysis, the recall, precision, accuracy and roc_auc_score is better for Logistic regression with smote applied.

In [ ]:
nb = GaussianNB()
nb.fit(trainX_oversampled, trainY_oversampled)
trainpredictY = nb.predict(trainX_oversampled)
testpredictY = nb.predict(testX)
print("Confusion Matrix for test data \n", confusion_matrix(testY, testpredictY))
print("Classification Report for test data\n", classification_report(testY, testpredictY))
# trainY = np.array(trainY).reshape(-1,1)
# trainpredictY = np.array(trainpredictY).reshape(-1,1)
# testY = np.array(testY).reshape(-1,1)
# testpredictY = np.array(testpredictY).reshape(-1,1)

auc = roc_auc_score(testY, testpredictY)
print('auc score {}'.format(auc))
fpr, tpr, thresholds = roc_curve(testY, testpredictY)
plt.plot(fpr, tpr)

> Hence Logisitc regression with oversampling(smote) technique for the given imbalanced data provides the best result in terms of accuracy, precision, recall, roc_auc_curve

> Note: Tried removing features with high correlation, but did not see improvement in model performnace, hence the features with high variance are also used to train the model.